# COMT - Single gene analysis in the AMP-PD data

- Project: COMT Single Gene analysis
- Version: Python/3.10.12
- Last Updated: 01-APRIL-2025
- Update Description: Clean code and adding some minor comments

## Notebook overview
In this notebook we performed regression analyses with COMT gene variants detected in WGS AMP-PD data using PLINK


## Description

__1. Description__

-  Getting Started
- Loading Python libraries
- Defining functions
- Set Paths
- Make working directory

__2. Installing packages__

__3. Copy over data__

__4. Create a covariate file with AMP-PD data__

__5. Remove related individuals and PCA__

__6. Annotation of the gene__

__7. Case/Control Analysis__

__8. Clinical scales Association Analyses__


## Loading Python libraries

In [2]:
# Use the os package to interact with the environment
import os
import sys

# Bring in Pandas for Dataframe functionality
import pandas as pd
from functools import reduce

# Bring some visualization functionality 
import seaborn as sns  

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

/tmp/ipykernel_121/4190448927.py:26: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Defining functions

In [3]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

## Set paths

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name @ `WORKSPACE_NAME`: {WORKSPACE_NAME}')
print(f'Billing Project @ `BILLING_PROJECT_ID`: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data @ `WORKSPACE_BUCKET`: {WORKSPACE_BUCKET}')
print('')

## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'path/'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'
AMP_RELEASE_GATK_PATH = os.path.join(AMP_RELEASE_PATH, 'gatk')
AMP_WGS_RELEASE_PATH = 'path/'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

## Make working directory

In [5]:
# Make a directory called "/home/jupyter/COMT_AMPPD/"
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

shell_do(f'mkdir -p {WORK_DIR}') # f' means f-string - contains expressions to execute the code


Executing: mkdir -p /home/jupyter/COMT_AMPPD


# Install Packages

## PLINK

In [10]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/


Install plink2. If this doesn't work, get the updated download link from https://www.cog-genomics.org/plink/2.0/, as this changes regularly. We use the download link for Linux AVX2 AMD.

In [13]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/plink2_linux_amd_avx2_20240806.zip
unzip -o plink2_linux_amd_avx2_20240806.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink2 is already installed in /home/jupyter/tools/


## ANNOVAR

In [18]:
%%capture
%%bash

# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

### ANNOVAR: Download sources of annotation


In [22]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20170905 humandb/


## RVTests

In [83]:
%%bash

#Install RVTESTS: Option 1 (~15min)
if test -e /home/jupyter/tools/rvtests; then

echo "rvtests is already installed"
else
echo "rvtests is not installed"

mkdir /home/jupyter/tools/rvtests
cd /home/jupyter/tools/rvtests

wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz 

tar -zxvf rvtests_linux64.tar.gz
fi

rvtests is already installed


In [91]:
# chmod to make sure you have permission to run the program
! chmod u+x /home/jupyter/tools/plink
! chmod u+x /home/jupyter/tools/plink2
! chmod 777 /home/jupyter/tools/rvtests/executable/rvtest

In [ ]:
%%bash

#Show installed tools
ls /home/jupyter/tools/

# Copy Over Files 

In [ ]:
#Check what is in the main AMP-PD release 3 release path
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {AMP_RELEASE_PATH}')

In [ ]:
#Check what is in the WGS release path
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {AMP_WGS_RELEASE_PATH}')

In [ ]:
# Check directory where AMP-PD data is
print("List available imputed genotype information in AMPPD")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {AMP_WGS_RELEASE_PLINK_PFILES}')

In [ ]:
##  Imputed genotype AMP-PD release 3 data
# COMT coordinates: chr22:19,941,371-19,969,975(GRCh38/hg38) https://www.genecards.org/cgi-bin/carddisp.pl?gene=COMT
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {AMP_WGS_RELEASE_PLINK_PFILES}/chr22.* {WORK_DIR}')


In [ ]:
#Copy other AMP-PD files (clinical info)
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_RELEASE_PATH}/amp_pd_case_control.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/Enrollment.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/Demographics.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/PD_Medical_History.csv  {WORK_DIR}')

In [ ]:
## refFlat
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/refFlat.txt.gz {WORK_DIR}')
!gzip -d {WORK_DIR}/refFlat.txt.gz

In [5]:
%%bash

cd $WORK_DIR

#wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
#gunzip -k hg38.fa.gz

# Create a covariate file with AMP-PD data

In [6]:
# load clinical information
pd_case_control_df = pd.read_csv(f'{WORK_DIR}/amp_pd_case_control.csv')

# visualize
#pd_case_control_df

In [7]:
# Keep columns of interest
pd_case_control_latest_df = pd_case_control_df[['participant_id', 'diagnosis_latest', 'case_control_other_latest']].copy()

# Rename Columns
pd_case_control_latest_df.columns = ['ID', 'LATEST_DX', 'CASE_CONTROL']


In [8]:
#Check case/control value counts
print(pd_case_control_latest_df['CASE_CONTROL'].value_counts())

CASE_CONTROL
Control    4337
Case       3538
Other      2933
Name: count, dtype: int64


In [9]:
# Add column for study origin
pd_case_control_latest_df['COHORT']= np.where(pd_case_control_latest_df.ID.str.contains("LB-"), "LBD",
                                    np.where(pd_case_control_latest_df.ID.str.contains("PP-"), "PPMI",
                                    np.where(pd_case_control_latest_df.ID.str.contains("PD-"), "PDBP",
                                    np.where(pd_case_control_latest_df.ID.str.contains("HB-"), "HBS",
                                    np.where(pd_case_control_latest_df.ID.str.contains("LC-"), "LCC",
                                    np.where(pd_case_control_latest_df.ID.str.contains("BF-"), "BIOFIND",
                                    np.where(pd_case_control_latest_df.ID.str.contains("SU-"), "SURE-PD3",
                                    np.where(pd_case_control_latest_df.ID.str.contains("SY-"), "STEADY-PD3", np.nan))))))))


In [10]:
# Drop duplicates
case_con_reduced = pd_case_control_latest_df.copy()
case_con_reduced.drop_duplicates(subset=['ID'], inplace=True)

In [11]:
#Assign case control for Plink
conditions = [
    (case_con_reduced['CASE_CONTROL'] == "Case"),
    (case_con_reduced['CASE_CONTROL'] == "Control")]
## Assign cases=2, controls=1, -9 Other (as used in PLINK)
choices = [2,1]
case_con_reduced['PHENO'] = np.select(conditions, choices, default=-9).astype(np.int64)

# set indices
case_con_reduced.reset_index(inplace=True)
case_con_reduced.drop(columns=["index"], inplace=True)

# Remove initial Pheno Column
case_con_reduced.drop(columns=['CASE_CONTROL'], inplace=True)


In [12]:
# Phenotype counts
print(case_con_reduced['PHENO'].value_counts(dropna=False))

PHENO
 1    4337
 2    3537
-9    2933
Name: count, dtype: int64


In [13]:
# Load Enrollment.csv
enrollment_df = pd.read_csv(f'{WORK_DIR}/Enrollment.csv')


In [14]:
# Keep columns of interest
enrollment_subset_df = enrollment_df[['participant_id', 'study_arm']].copy()

# Rename columns
enrollment_subset_df.columns = ['ID', 'ENROLL_STUDY_ARM']
enrollment_subset_df.head()

# Drop duplicates
enrollment_subset_df.drop_duplicates(subset=['ID'], keep='first', inplace=True)

In [15]:
# load demographic data
demographics_df = pd.read_csv(f'{WORK_DIR}/Demographics.csv')

In [16]:
# Rename Columns
demographics_df.rename(columns = {'participant_id':'ID'}, inplace = True)
demographics_df.rename(columns = {'age_at_baseline':'BASELINE_AGE'}, inplace = True)
demographics_df.rename(columns = {'race':'RACE'}, inplace = True)
demographics_df.rename(columns = {'ethnicity':'ETHNICITY'}, inplace = True)

In [17]:
# Sort by visit month and Drop Duplicates
demographics_baseline_df = demographics_df \
.sort_values('visit_month', ascending=True) \
.drop_duplicates('ID').sort_index()

In [18]:
# Merge last diagnostic with diagnostic at enrollement
demographics_df_casecon = demographics_df.merge(case_con_reduced, on='ID', how='outer')

In [19]:
#Recode Sex Columns
conditions = [
     (demographics_df_casecon['sex'] == "Male"),
     (demographics_df_casecon['sex'] == "Female")]

# 1=Male; 2=Female
choices = [1,2]

# Create new column Sex
demographics_df_casecon['SEX'] = np.select(conditions, choices, default=None).astype(np.int64)

# Remove previous Sex column
demographics_df_casecon.drop(columns=['sex'], inplace=True)

In [20]:
## Keep only columns of interest
demographics_df_casecon_toKeep = demographics_df_casecon[['ID', 'PHENO', 'SEX', 'RACE',
                                                          'ETHNICITY','BASELINE_AGE', 'LATEST_DX',
                                                          'COHORT']].copy()

In [21]:
# Merge Pheno with demograhic data
enrollment_pheno_df = demographics_df_casecon_toKeep.merge(enrollment_subset_df, on='ID', how='outer')

# Create FID column
enrollment_pheno_df['FID'] = enrollment_pheno_df['ID'].values

# rename ID as IID 
enrollment_pheno_df.rename(columns = {'ID':'IID'}, inplace = True)

# Order columns
reorder_enrollment_pheno_df = enrollment_pheno_df[['FID', 'IID', 'PHENO',
                                                  'SEX', 'RACE','ETHNICITY', 'BASELINE_AGE', 'LATEST_DX',
                                                  'COHORT', 'ENROLL_STUDY_ARM']].copy()

In [24]:
#Remove individuals from the genetic registry cohorts and other enrollment categories
#We only want to keep  controls who were originally enrolled as controls, and PD cases originally enrolled as cases
#Will exclude prodromal, SWEDD and unknown individuals also
#Individuals enrolled as disease control but the latest diagnosis is PD will also be excluded

#Keep only individuals enrolled as Healthy Control/Disease Control or PD.
filtered_enrollment_pheno_df = reorder_enrollment_pheno_df.copy()
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[filtered_enrollment_pheno_df['ENROLL_STUDY_ARM'].isin(['Disease Control', 
                                                                                                                   'Healthy Control', 
                                                                                                                  'PD'])]
#Now remove individuals with PHENO of -9 (keep only individuals with PHENO of 1 or 2)
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[filtered_enrollment_pheno_df['PHENO'].isin([1,2])]

#Now remove individuals who were enrolled with the opposite diagnosis, i.e. individuals who were enrolled as controls but have a latest diagnosis of PD
filtered_enrollment_pheno_df = filtered_enrollment_pheno_df[((filtered_enrollment_pheno_df['PHENO'] == 2) & (filtered_enrollment_pheno_df['ENROLL_STUDY_ARM'] == 'PD')) | (filtered_enrollment_pheno_df['PHENO'] == 1)]


#Check value counts again
filtered_enrollment_pheno_df.groupby(['PHENO', 'ENROLL_STUDY_ARM']).size().reset_index(name='counts')


In [25]:
#Save file - this is intermediate not final covariate file
#This includes all ancestries
filtered_enrollment_pheno_df.to_csv(f'{WORK_DIR}/COVS_temp.txt', index=False, sep='\t', na_rep='NA')

## Import GenoTools ancestry

This is needed for the ancestry labels. Need to do this before running the PCA as the PCs should be generated just in the population/group of interest.

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv {WORK_DIR}')

In [26]:
#Read covariate file to get ancestry groups
ancestries = pd.read_csv(f'{WORK_DIR}/AMPPD_v3_COV_wPHENOS_wGENOTOOLS.csv')

In [27]:
#This file has a lot of the diagnosis info that we have generated ourselves earlier.
#We just want the ancestry label (the last column)

ancestries_selected = ancestries[['ID', 'label']]

In [28]:
#Filter for ancestry of interest
#Change this to your ancestry of interest
ancestry_toKeep = 'EUR'

ancestries_keep = ancestries_selected.copy()
ancestries_keep = ancestries_selected[ancestries_selected['label'] == ancestry_toKeep]

In [29]:
#Check the value counts again - this should just be the ancestry you are interested in
ancestries_keep.label.value_counts()

label
EUR    8607
Name: count, dtype: int64

In [30]:
#Merge covariate file with the ancestry file
#Inner join - meaning we only want to keep individuals in both dataframes
#So this will keep just individuals in the selected ancestry group
merged = pd.merge(filtered_enrollment_pheno_df, ancestries_keep, left_on = 'IID', right_on = 'ID', how = 'inner')


In [31]:
#Write list of individuals from the ancestry group of interest to keep from the genetic dataset
#We need to do this before generating PCs
individuals_toKeep = merged[['FID', 'IID']]


In [32]:
#Export file
#Note this still has related individuals
#We will remove related individuals at the next step
individuals_toKeep.to_csv('/home/jupyter/COMT_AMPPD/individuals_toKeep.EUR_enroll.txt', index=False, sep='\t')

# Remove related individuals

In [ ]:
#From the workspace bucket, copy over the file with EUR related individuals to remove (just one individual from each pair)
#Make sure to copy over the file for the correct ancestry
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {WORKSPACE_BUCKET}/toRemove_1stand2ndDegree_Relateds_EUR.txt {WORK_DIR}')

In [33]:
# Remove related individuals and keep only PD and controls in the correct enrollment categories
# Used make-bed not make-pgen because there is a bug in plink2 and it doesn't write the pgen file https://groups.google.com/g/plink2-users/c/z_ZVhacpnts/m/byzJxGl5AwAJ
# Have included max-alleles 2 because plink cannot write bim file with multialleleic variants
WORK_DIR = f'/home/jupyter/COMT_AMPPD'
! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr22 \
--keep {WORK_DIR}/individuals_toKeep.EUR_enroll.txt \
--remove {WORK_DIR}/toRemove_1stand2ndDegree_Relateds_EUR.txt \
--make-bed \
--max-alleles 2 \
--out {WORK_DIR}/chr22_nonrelated_enroll

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.log.
Options in effect:
  --keep /home/jupyter/COMT_AMPPD/individuals_toKeep.EUR_enroll.txt
  --make-bed
  --max-alleles 2
  --out /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll
  --pfile /home/jupyter/COMT_AMPPD/chr22
  --remove /home/jupyter/COMT_AMPPD/toRemove_1stand2ndDegree_Relateds_EUR.txt

Start time: Tue Apr  1 14:43:23 2025
7445 MiB RAM detected, ~6123 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
10418 samples (0 females, 0 males, 10418 ambiguous; 10418 founders) loaded from
/home/jupyter/COMT_AMPPD/chr22.psam.
1938650 out of 2159838 variants loaded from
/home/jupyter/COMT_AMPPD/chr22.pvar.
Note: No phenotype data present.
--keep: 5174 samples remaining.
--remove: 5086 samples remaining.
5086 samples (0 females, 0 males, 5086

In [ ]:
# Download from workspace bucket 
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORKSPACE_BUCKET}/AMPPD_EUR.COVS.txt {WORK_DIR}')

## Update sex and pheno info in plink genetic files

In [35]:
# Update sex and phenotype in plink files, using the AMPPD_EUR.COVS.txt file that we made
#-update-sex and column number tells plink where to look for the sex information in the file

WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/chr22_nonrelated_enroll \
--update-sex {WORK_DIR}/AMPPD_EUR.COVS.txt col-num=5 \
--pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
--pheno-name PHENO \
--make-pgen \
--out {WORK_DIR}/chr22_nonrelated_enroll.updated

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated.log.
Options in effect:
  --bfile /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll
  --make-pgen
  --out /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated
  --pheno /home/jupyter/COMT_AMPPD/AMPPD_EUR.COVS.txt
  --pheno-name PHENO
  --update-sex /home/jupyter/COMT_AMPPD/AMPPD_EUR.COVS.txt col-num=5

Start time: Tue Apr  1 14:45:16 2025
7445 MiB RAM detected, ~6079 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (0 females, 0 males, 5086 ambiguous; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.fam.
1938650 variants loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.bim.
1 binary phenotype loaded (2251 cases, 2835 controls).
--update-sex: 5086 samples updated, 1 ID not presen

# Extract and annotate the gene

## Extract the region using PLINK

In [39]:
## update the variant IDs
! /home/jupyter/tools/plink2 \
--pfile /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated \
--fa /home/jupyter/hg38.fa \
--memory 25000 \
--set-all-var-ids "chr@:#:\$r:\$a" \
--new-id-max-allele-len 999 --sort-vars \
--make-pgen \
--out /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted.log.
Options in effect:
  --fa /home/jupyter/hg38.fa
  --make-pgen
  --memory 25000
  --new-id-max-allele-len 999
  --out /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted
  --pfile /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated
  --set-all-var-ids chr@:#:$r:$a
  --sort-vars

Start time: Fri Mar 14 15:51:08 2025
7445 MiB RAM detected, ~6206 available; reserving 6142 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated.psam.
1938650 variants loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated.pvar.
1 binary phenotype loaded (2251 cases, 2835 controls).
--fa: Length scraped for 1 contig.
W

In [40]:
## Chromosome 22: 19,941,371-19,969,975
## extract region using plink and make pgen files
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr22_nonrelated_enroll.updated_formatted \
--chr 22 \
--from-bp 19941371 \
--to-bp 19969975 \
--max-alleles 2 \
--mac 2 \
--geno 0.05 \
--make-pgen \
--out {WORK_DIR}/COMT

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/COMT.log.
Options in effect:
  --chr 22
  --from-bp 19941371
  --geno 0.05
  --mac 2
  --make-pgen
  --max-alleles 2
  --out /home/jupyter/COMT_AMPPD/COMT
  --pfile /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted
  --to-bp 19969975

Start time: Fri Mar 14 15:52:04 2025
7445 MiB RAM detected, ~6061 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted.psam.
1938650 variants loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted.pvar.
1 binary phenotype loaded (2251 cases, 2835 controls).
Calculating allele frequencies... done.
--geno: 3 variants removed due to missing genotype data.
1176 variants

In [41]:
## Extract COMT region using plink and make also binary files
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--pfile {WORK_DIR}/chr22_nonrelated_enroll.updated_formatted \
--chr 22 \
--from-bp 19941371 \
--to-bp 19969975 \
--max-alleles 2 \
--mac 2 \
--geno 0.05 \
--make-bed \
--out {WORK_DIR}/COMT


PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/COMT.log.
Options in effect:
  --chr 22
  --from-bp 19941371
  --geno 0.05
  --mac 2
  --make-bed
  --max-alleles 2
  --out /home/jupyter/COMT_AMPPD/COMT
  --pfile /home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted
  --to-bp 19969975

Start time: Fri Mar 14 15:52:05 2025
7445 MiB RAM detected, ~6081 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted.psam.
1938650 variants loaded from
/home/jupyter/COMT_AMPPD/chr22_nonrelated_enroll.updated_formatted.pvar.
1 binary phenotype loaded (2251 cases, 2835 controls).
Calculating allele frequencies... done.
--geno: 3 variants removed due to missing genotype data.
1176 variants 

## Turn binary files into VCF

In [42]:
## Turn binary files into VCF
WORK_DIR = f'/home/jupyter/COMT_AMPPD'
! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/COMT \
--recode vcf id-paste=iid \
--out {WORK_DIR}/COMT

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/COMT.log.
Options in effect:
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --export vcf id-paste=iid
  --out /home/jupyter/COMT_AMPPD/COMT

Start time: Fri Mar 14 15:53:56 2025
7445 MiB RAM detected, ~6095 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.fam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.bim.
1 binary phenotype loaded (2251 cases, 2835 controls).
--export vcf to /home/jupyter/COMT_AMPPD/COMT.vcf ... 0%0%1%1%2%2%3%3%4%4%5%5%6%6%7%7%8%8%9%9%10%10%11%11%12%12%13%13%14%14%15%15%16%16%17%17%18%18%19%19%20%20%21%21%22%22%23%23%24%

In [43]:
### Bgzip and Tabix (zip and index the file)

! bgzip -f {WORK_DIR}/COMT.vcf
! tabix -f -p vcf {WORK_DIR}/COMT.vcf.gz 

## Annotate using ANNOVAR

I have included with the dbnsfp47a database which includes the CADD scores. The ANNOVAR tutorial uses v3.0 of the dbNSFP database (https://annovar.openbioinformatics.org/en/latest/user-guide/startup/#a-useful-tutorial) but dbNSFP v4.7 is the latest version (see http://database.liulab.science/dbNSFP#version). The 4.7a version is the academic version which includes CADD, the commercial version does not include CADD (https://www.biostars.org/p/9573857/).

In [ ]:
## annotate using ANNOVAR
! perl /home/jupyter/tools/annovar/table_annovar.pl {WORK_DIR}/COMT.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out {WORK_DIR}/COMT.annovar \
-remove -protocol refGene,clinvar_20170905 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput

In [45]:
# Read in ANNOVAR multianno file
gene = pd.read_csv(f'{WORK_DIR}/COMT.annovar.hg38_multianno.txt', sep = '\t')
    
#Filter for the correct gene name (sometimes other genes are also included)
gene = gene[gene['Gene.refGene'] == 'COMT']
    
#Print number of variants in the different categories
results = [] 

intronic = gene[gene['Func.refGene']== 'intronic']
upstream = gene[gene['Func.refGene']== 'upstream']
downstream = gene[gene['Func.refGene']== 'downstream']
utr5 = gene[gene['Func.refGene']== 'UTR5']
utr3 = gene[gene['Func.refGene']== 'UTR3']
splicing = gene[gene['Func.refGene']== 'splicing']
exonic = gene[gene['Func.refGene']== 'exonic']
stopgain = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stopgain')]
stoploss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'stoploss')]
startloss = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'startloss')]
frameshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift deletion')]
frameshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'frameshift insertion')]
nonframeshift_deletion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift deletion')]
nonframeshift_insertion = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonframeshift insertion')]
coding_nonsynonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_synonymous = gene[(gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] == 'synonymous SNV')]
        
print("AMP-PD")
print('Total variants: ', len(gene))
print("Intronic: ", len(intronic))
print("Upstream: ", len(upstream))
print("Downstream: ", len(downstream))
print('UTR3: ', len(utr3))
print('UTR5: ', len(utr5))
print("Splicing: ", len(splicing))
print("Total exonic: ", len(exonic))
print("Stopgain: ", len(stopgain))
print("Stoploss: ", len(stoploss))
print("Startloss: ", len(startloss))
print("Frameshift deletion: ", len(frameshift_deletion))
print("Frameshift insertion: ", len(frameshift_insertion))
print("Non-frameshift insertion: ", len(nonframeshift_insertion))
print("Non-frameshift deletion: ", len(nonframeshift_deletion))
print('Synonymous: ', len(coding_synonymous))
print("Nonsynonymous: ", len(coding_nonsynonymous))
results.append((gene, intronic, upstream, downstream, utr3, utr5, splicing,exonic,stopgain,stoploss,startloss, frameshift_deletion,frameshift_insertion,nonframeshift_deletion,nonframeshift_insertion,coding_synonymous, coding_nonsynonymous))
print('\n')
    
## For rvtests
    
# Potential functional: These are variants annotated as frameshift, nonframeshift, startloss, stoploss, stopgain, splicing, missense, exonic, UTR5, UTR3, upstream (-100bp), downstream (+100bp), or ncRNA. 
potentially_functional = gene[gene['Func.refGene'] != 'intronic']
# Coding: These are variants annotated as frameshift, nonframeshift, startloss, stoploss, stopgain, splicing, or missense.
coding_variants = gene[(gene['Func.refGene'] == 'splicing') | (gene['Func.refGene'] == 'exonic') & (gene['ExonicFunc.refGene'] != 'synonymous SNV')]
# Loss of function: These are variants annotated as frameshift, startloss,stopgain, or splicing.
loss_of_function = gene[(gene['Func.refGene'] == 'splicing') | (gene['ExonicFunc.refGene'] == 'stopgain') | (gene['ExonicFunc.refGene'] == 'startloss') | (gene['ExonicFunc.refGene'] == 'frameshift deletion') | (gene['ExonicFunc.refGene'] == 'frameshift insertion')]
    
# Save in PLINK format
variants_toKeep = potentially_functional[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep.to_csv(f'{WORK_DIR}/COMT.potentially_functional.variantstoKeep.txt', sep="\t", index=False, header=False)

variants_toKeep2 = coding_variants[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep2.to_csv(f'{WORK_DIR}/COMT.coding_variants.variantstoKeep.txt', sep="\t", index=False, header=False)

variants_toKeep3 = loss_of_function[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep3.to_csv(f'{WORK_DIR}/COMT.loss_of_function.variantstoKeep.txt', sep="\t", index=False, header=False)
    
# For assoc
    
# These are all exonic variants
exonic = gene[gene['Func.refGene'] == 'exonic']
    
# Save in PLINK format
variants_toKeep4 = exonic[['Chr', 'Start', 'End', 'Gene.refGene']].copy()
variants_toKeep4.to_csv(f'{WORK_DIR}/COMT.exonic.variantstoKeep.txt', sep="\t", index=False, header=False)

AMP-PD
Total variants:  491
Intronic:  444
Upstream:  0
Downstream:  0
UTR3:  20
UTR5:  14
Splicing:  0
Total exonic:  13
Stopgain:  0
Stoploss:  0
Startloss:  0
Frameshift deletion:  0
Frameshift insertion:  0
Non-frameshift insertion:  0
Non-frameshift deletion:  1
Synonymous:  8
Nonsynonymous:  4




## Burden Analyses using RVTests

In [46]:
#Loop over 3 variant classes, extract variants and create the right format for RVTest
variant_classes = ['potentially_functional', 'coding_variants','loss_of_function']

WORK_DIR = f'/home/jupyter/COMT_AMPPD'

for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running plink to extract {variant_class} variants for AMP-PD')
    
    #Extract relevant variants
    ! /home/jupyter/tools/plink2 \
    --pfile {WORK_DIR}/COMT \
    --extract range {WORK_DIR}/COMT.{variant_class}.variantstoKeep.txt \
    --recode vcf-iid \
    --out {WORK_DIR}/AMPPD_COMT.{variant_class}
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running bgzip and tabix for {variant_class} variants for AMP-PD')
        
    ## Bgzip and Tabix (zip and index the file)
    ! bgzip -f {WORK_DIR}/AMPPD_COMT.{variant_class}.vcf
    ! tabix -f -p vcf {WORK_DIR}/AMPPD_COMT.{variant_class}.vcf.gz

Running plink to extract potentially_functional variants for AMP-PD
PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/AMPPD_COMT.potentially_functional.log.
Options in effect:
  --export vcf-iid
  --extract range /home/jupyter/COMT_AMPPD/COMT.potentially_functional.variantstoKeep.txt
  --out /home/jupyter/COMT_AMPPD/AMPPD_COMT.potentially_functional
  --pfile /home/jupyter/COMT_AMPPD/COMT

Start time: Fri Mar 14 15:56:41 2025
Note: --export 'vcf-iid' modifier is deprecated.  Use 'vcf' + 'id-paste=iid'.
7445 MiB RAM detected, ~6020 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.psam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.pvar.
1 binary phenotype loaded (2251 cases, 2835 controls).
--extract bed1: 456

In [ ]:
# Run RVTest for each variant class

variant_classes = ['potentially_functional', 'coding_variants','loss_of_function']

for variant_class in variant_classes:
    
    # Print the command to be executed (for debugging purposes)
    print(f'Running RVtests for {variant_class} variants for AMP-PD')
        
     ## RVtests with covariates 
    #Make sure the pheno and covariate file starts with the first 5 columns: fid, iid, fatid, matid, sex
    #The pheno-name flag only works when the pheno/covar file is structured properly
    ! /home/jupyter/tools/rvtests/executable/rvtest --noweb --hide-covar \
    --out {WORK_DIR}/AMPPD_COMT.burden.{variant_class} \
    --kernel skat,skato \
    --inVcf {WORK_DIR}/AMPPD_COMT.{variant_class}.vcf.gz \
    --pheno {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --pheno-name PHENO \
    --gene COMT \
    --geneFile {WORK_DIR}/refFlat.txt \
    --covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
    --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5


In [48]:
# Look at results for potentially functional variants 
! cat {WORK_DIR}/AMPPD_COMT.burden.potentially_functional.Skat.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	Pvalue	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
COMT	22:19962552-19969975,22:19951517-19969975,22:19950901-19969975,22:19941771-19969975,22:19941771-19969975	5086	46	46	154675	0.0597463	10000	10000	154675	713	0	0.0713


In [49]:
! cat {WORK_DIR}/AMPPD_COMT.burden.potentially_functional.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
COMT	22:19962552-19969975,22:19951517-19969975,22:19950901-19969975,22:19941771-19969975,22:19941771-19969975	5086	46	46	77337.6	0	0.108518


In [50]:
## Look at results for coding variants 

! cat {WORK_DIR}/AMPPD_COMT.burden.coding_variants.SkatO.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	rho	Pvalue
COMT	22:19962552-19969975,22:19951517-19969975,22:19950901-19969975,22:19941771-19969975,22:19941771-19969975	5086	4	4	84.8181	1	0.78128


In [51]:
! cat {WORK_DIR}/AMPPD_COMT.burden.coding_variants.Skat.assoc

Gene	RANGE	N_INFORMATIVE	NumVar	NumPolyVar	Q	Pvalue	NumPerm	ActualPerm	Stat	NumGreater	NumEqual	PermPvalue
COMT	22:19962552-19969975,22:19951517-19969975,22:19950901-19969975,22:19941771-19969975,22:19941771-19969975	5086	4	4	199.981	0.82832	10000	1097	199.981	1000	0	0.911577


In [ ]:
# Save to workspace bucket (move from VM to workspace bucket)
variant_classes = ['potentially_functional', 'coding_variants','loss_of_function']

for variant_class in variant_classes:
    shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/AMPPD_COMT.burden.{variant_class}.*.assoc {WORKSPACE_BUCKET}/COMT_AMPPD/')

# Case/Control Analysis


## All variants

### glm

In [52]:
# Check format
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

!head {WORK_DIR}/COMT.bim

22	chr22:19941423:G:A	0	19941423	A	G
22	chr22:19941469:G:A	0	19941469	A	G
22	chr22:19941499:G:A	0	19941499	A	G
22	chr22:19941618:T:TG	0	19941618	TG	T
22	chr22:19941620:G:A	0	19941620	A	G
22	chr22:19941630:C:T	0	19941630	T	C
22	chr22:19941640:C:T	0	19941640	T	C
22	chr22:19941662:G:T	0	19941662	T	G
22	chr22:19941740:C:T	0	19941740	T	C
22	chr22:19941872:G:C	0	19941872	C	G


In [44]:
#Run glm with covariates. We used a0-ref to do the regression on the right allele for rs4680.
#From the 16 Feb 2018 (alpha 2) entry in the version history: 
#"Also, --glm now defaults to regressing on minor instead of ALT allele dosages 
#(this can be overridden with 'a0-ref')."


WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/COMT \
--glm \
--adjust \
--ci 0.95 \
--mac 2 \
--geno 0.05 \
--maf 0.01 \
--hwe 0.0001 \
--covar {WORK_DIR}/AMPPD_EUR.COVS.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/AMPPD_COMT.allvariants

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/AMPPD_COMT.allvariants.log.
Options in effect:
  --adjust
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --ci 0.95
  --covar /home/jupyter/COMT_AMPPD/AMPPD_EUR.COVS.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --geno 0.05
  --glm
  --hwe 0.0001
  --mac 2
  --maf 0.01
  --out /home/jupyter/COMT_AMPPD/AMPPD_COMT.allvariants

Start time: Tue Apr  1 14:51:30 2025
7445 MiB RAM detected, ~6059 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.fam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.bim.
1 binary phenotype loaded (2251 cases, 2835 controls).
7 covariates loaded from /home/jupyter/COMT_AMPPD/AMPPD_EUR.COVS.txt.
--covar-variance-

In [38]:
results_glm = pd.read_csv(f'{WORK_DIR}/AMPPD_COMT.allvariants.PHENO1.glm.logistic.hybrid', delim_whitespace=True)
results_glm_add=results_glm[(results_glm['TEST']=='ADD')]


In [43]:
results_glm_adj = pd.read_csv(f'{WORK_DIR}/AMPPD_COMT.allvariants.PHENO1.glm.logistic.hybrid.adjusted', delim_whitespace=True)


### Add more info to the glm results

In [25]:
#--recode A creates a new text fileset, showing each variant in each case and control for the minor allele (A).
# Also extract the significant variants 
! /home/jupyter/tools/plink \
--bfile {WORK_DIR}/COMT \
--recode A \
--out {WORK_DIR}/AMPPD_COMT.AllVariants.adj

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/AMPPD_COMT.AllVariants.adj.log.
Options in effect:
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --out /home/jupyter/COMT_AMPPD/AMPPD_COMT.AllVariants.adj
  --recode A

7445 MB RAM detected; reserving 3722 MB for main workspace.
502 variants loaded from .bim file.
5086 people (2803 males, 2283 females) loaded from .fam.
5086 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 5086 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999957.
502 variants and 5086 people pass filters and QC.
Among remaining p

In [26]:
recode = pd.read_csv(f'{WORK_DIR}/AMPPD_COMT.AllVariants.adj.raw', delim_whitespace=True)


In [27]:
# Make a list from the column names
column_names = recode.columns.tolist()

# Drop the first 6 columns to keep the variants 
variants = column_names[6:]

print(f'Number of variants in AMPPD for COMT: {len(variants)}')


Number of variants in AMPPD for COMT: 502


In [65]:
# Pre-filter the dataset
cases_data = recode[recode['PHENOTYPE'] == 2]
controls_data = recode[recode['PHENOTYPE'] == 1]
total_cases = cases_data.shape[0]
total_controls = controls_data.shape[0]
results = []


for variant in variants:
        hom_cases = (cases_data[variant] == 2).sum()
        het_cases = (cases_data[variant] == 1).sum()
        hom_ref_cases = (cases_data[variant] == 0).sum()  # Homozygous reference genotype
        missing_cases = total_cases - (hom_cases + het_cases + hom_ref_cases)  # Missing data count
        freq_cases = (2 * hom_cases + het_cases) / (2 * (total_cases - missing_cases))  # Adjust for missing data in denominator

        ## For controls
        hom_controls = (controls_data[variant] == 2).sum()
        het_controls = (controls_data[variant] == 1).sum()
        hom_ref_controls = (controls_data[variant] == 0).sum()  # Homozygous reference genotype
        missing_controls = total_controls - (hom_controls + het_controls + hom_ref_controls)  # Missing data count
        freq_controls = (2 * hom_controls + het_controls) / (2 * (total_controls - missing_controls))  # Adjust for missing data in denominator
    
        # Append results in dictionary format
        results.append({
            'Variant': variant,
            'Hom Cases': hom_cases,
            'Het Cases': het_cases,
            'Hom Ref Cases': hom_ref_cases,
            'Missing Cases': missing_cases,
            'Total Cases': total_cases,
            'Carrier Freq in Cases': freq_cases,
            'Hom Controls': hom_controls,
            'Het Controls': het_controls,
            'Hom Ref Controls': hom_ref_controls,
            'Missing Controls': missing_controls,
            'Total Controls': total_controls,
            'Carrier Freq in Controls': freq_controls
        })

# Return
df_results = pd.DataFrame(results)
df_results['ID'] = df_results['Variant'].apply(lambda x: x.rsplit('_', 1)[0])



In [144]:
#Merge with the glm file
results_glm_add_merge = results_glm_add[['ID','A1','OR','L95','U95','P']]
merged = pd.merge(df_results, results_glm_add_merge, on='ID', how='right')

In [71]:
## to CSV
merged.to_csv(f'{WORK_DIR}/AMPPD_COMT_AllVariants.glm.txt', sep = '\t', index=False)

In [ ]:
# Save the final results to workspace bucket (move from VM to workspace bucket)
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR}/AMPPD_COMT_AllVariants.glm.txt {WORKSPACE_BUCKET}/COMT_AMPPD/AMPPD_COMT_allvariants_minusintronic.glm.txt')


# Clinical scales Association Analysis

## MOCA

In [ ]:
# Download clinical data: MOCA scale
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/MOCA.csv {WORK_DIR}')

In [54]:
# Also re-use the AMPPD covariate file used before
AMPPD_COV = pd.read_csv(f'{WORK_DIR}/AMPPD_EUR.COVS.txt',sep='\t')

In [35]:
AMPPD_COV['PHENO'].value_counts()

PHENO
1    2835
2    2251
Name: count, dtype: int64

In [73]:
# Another source of data is the Demographic.csv file, to obtain the education level. Note that we will use baseline
education = pd.read_csv(f'{WORK_DIR}/Demographics.csv')
education = education[['participant_id','visit_month','visit_name','education_level_years']]
education_M0 = education[education['visit_month']==0]
education_M0 = education_M0[['participant_id','education_level_years']]
education_M0.columns = ['IID','education_level_years']

In [74]:
# Now read the MOCA.csv file and format it to only contain baseline and the selected columns
moca_df = pd.read_csv(f'{WORK_DIR}/MOCA.csv')
moca_df = moca_df[['participant_id','visit_month','moca_total_score']]
moca_m0_df = moca_df[moca_df['visit_month']==0] 
moca_m0_df.columns = ['IID','visit_name','moca_total_score']

In [75]:
# Merge with the COV file and then include also the education data
AMPPD_COV_moca = pd.merge(moca_m0_df,AMPPD_COV,on="IID")
AMPPD_COV_moca_education = pd.merge(AMPPD_COV_moca,education_M0,on='IID')

In [79]:
# Check the values of PHENO
AMPPD_COV_moca_education['PHENO'].value_counts()

PHENO
2    1349
1     495
Name: count, dtype: int64

In [80]:
AMPPD_COV_moca_education = AMPPD_COV_moca_education.drop_duplicates(subset=['IID'])

In [81]:
# Select the columns
AMPPD_COV_moca_education = AMPPD_COV_moca_education[['IID','IID','AGE','PHENO','SEX','education_level_years','moca_total_score','PC1','PC2','PC3','PC4','PC5']]

In [82]:
# Rename the columns
AMPPD_COV_moca_education.columns = ['FID','IID','AGE','PHENO','SEX','Education_years','moca_total_score','PC1','PC2','PC3','PC4','PC5']

In [83]:
# Reformat education years
AMPPD_COV_moca_education['Education_years'] = AMPPD_COV_moca_education['Education_years'].replace({'Unknown': '-9'})
AMPPD_COV_moca_education['Education_years'] = AMPPD_COV_moca_education['Education_years'].replace({'Less than 12 years': '0'})
AMPPD_COV_moca_education['Education_years'] = AMPPD_COV_moca_education['Education_years'].replace({'12-16 years': '1'})
AMPPD_COV_moca_education['Education_years'] = AMPPD_COV_moca_education['Education_years'].replace({'Greater than 16 years': '2'})
AMPPD_COV_moca_education['Education_years'].value_counts()

Education_years
1     1041
2      498
-9     258
0       39
Name: count, dtype: int64

In [84]:
# We are testing the association of the variants with MOCA total score on PD patients
AMPPD_COV_moca_education_EP=AMPPD_COV_moca_education[AMPPD_COV_moca_education['PHENO']==2]

In [85]:
# Save on a file 
AMPPD_COV_moca_education_EP.to_csv(f'{WORK_DIR}/COVAR_MOCA_EP.txt',sep="\t",index=None)

In [45]:
# Test the association for all variants of the gene detected with MOCA scale total score
WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/COMT \
--glm a0-ref\
--pheno {WORK_DIR}/COVAR_MOCA_EP.txt \
--pheno-name moca_total_score \
--adjust \
--mac 2 \
--hwe 0.0001 \
--geno 0.05 \
--maf 0.01 \
--covar {WORK_DIR}/COVAR_MOCA_EP.txt \
--covar-name SEX,AGE,Education_years,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/MOCA_COMT_All_variants

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/MOCA_COMT_All_variants.log.
Options in effect:
  --adjust
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --covar /home/jupyter/COMT_AMPPD/COVAR_MOCA_EP.txt
  --covar-name SEX,AGE,Education_years,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --geno 0.05
  --glm a0-ref
  --hwe 0.0001
  --mac 2
  --maf 0.01
  --out /home/jupyter/COMT_AMPPD/MOCA_COMT_All_variants
  --pheno /home/jupyter/COMT_AMPPD/COVAR_MOCA_EP.txt
  --pheno-name moca_total_score

Start time: Tue Apr  1 14:52:00 2025
7445 MiB RAM detected, ~6059 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.fam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.bim.
1 quantitative phenotype loaded (1341 values).


In [88]:
glm_MOCA_AllVariants = pd.read_csv(f'{WORK_DIR}/MOCA_COMT_All_variants.moca_total_score.glm.linear', delim_whitespace=True)
glm_MOCA_AllVariants_ADD = glm_MOCA_AllVariants[glm_MOCA_AllVariants['TEST']=="ADD"]

In [90]:
# Also check the P corrected for multiple comparison
glm_MOCA_AllVariants_adjusted= pd.read_csv(f'{WORK_DIR}/MOCA_COMT_All_variants.moca_total_score.glm.linear.adjusted', delim_whitespace=True)

In [89]:
# Save the results of the glm to a file
glm_MOCA_AllVariants_ADD.to_csv(f'{WORK_DIR}/MOCA_COMT.AllVariants.csv')

## MDS UPDRS III


In [ ]:
# Download clinical data: MDS-UPDRS scale part III
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/MDS_UPDRS_Part_III.csv {WORK_DIR}')

In [86]:
# Re-use the COV file (demographics of case-control samples)
AMPPD_COV = pd.read_csv(f'{WORK_DIR}/AMPPD_EUR.COVS.txt',sep='\t')

In [99]:
# Now read the MDS_UPDRS_PART_III.csv file and 
# format it to only contain baseline and the selected columns
mds_updrs_iii = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_Part_III.csv')
mds_updrs_iii = mds_updrs_iii[['participant_id','visit_month','mds_updrs_part_iii_summary_score']]
mds_updrs_iii_m0 = mds_updrs_iii[mds_updrs_iii['visit_month']==0] 
mds_updrs_iii_m0.columns = ['IID','visit_month','MDS_UPDRS_III']


In [100]:
# Merge with the COV file and check the sample size
AMPPD_COV_UPDRSIII = pd.merge(mds_updrs_iii_m0,AMPPD_COV,on="IID")
AMPPD_COV_UPDRSIII['PHENO'].value_counts()

PHENO
2    2193
1     646
Name: count, dtype: int64

In [101]:
# Select the columns
AMPPD_COV_UPDRSIII = AMPPD_COV_UPDRSIII[['IID','IID','AGE','PHENO','SEX','MDS_UPDRS_III','PC1','PC2','PC3','PC4','PC5']]

In [102]:
# Rename the columns
AMPPD_COV_UPDRSIII.columns = ['FID','IID','AGE','PHENO','SEX','MDS_UPDRS_III','PC1','PC2','PC3','PC4','PC5']

In [103]:
# Reformat the interest column to numeric
AMPPD_COV_UPDRSIII["MDS_UPDRS_III"] = pd.to_numeric(AMPPD_COV_UPDRSIII["MDS_UPDRS_III"])

In [104]:
# Split the data on PD and HC
AMPPD_COV_UPDRSIII_EP=AMPPD_COV_UPDRSIII[AMPPD_COV_UPDRSIII['PHENO']==2]

In [105]:
# Check NAs and fill with -9
AMPPD_COV_UPDRSIII_EP['MDS_UPDRS_III'] = AMPPD_COV_UPDRSIII_EP['MDS_UPDRS_III'].fillna(-9)

/tmp/ipykernel_79/2200475102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AMPPD_COV_UPDRSIII_EP['MDS_UPDRS_III'] = AMPPD_COV_UPDRSIII_EP['MDS_UPDRS_III'].fillna(-9)


In [106]:
# Save to a file
AMPPD_COV_UPDRSIII_EP.to_csv(f'{WORK_DIR}/COVAR_MDS_UPDRS_III_EP.txt',sep="\t",index=None)

In [46]:
# Test the association for all variants of the gene detected with UPDRSIII scale total score

WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/COMT \
--glm a0-ref\
--pheno {WORK_DIR}/COVAR_MDS_UPDRS_III_EP.txt \
--pheno-name MDS_UPDRS_III \
--adjust \
--mac 2 \
--geno 0.05 \
--hwe 0.0001 \
--maf 0.01 \
--covar {WORK_DIR}/COVAR_MDS_UPDRS_III_EP.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/MDS_UPDRS_III_COMT_All_variants

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/MDS_UPDRS_III_COMT_All_variants.log.
Options in effect:
  --adjust
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --covar /home/jupyter/COMT_AMPPD/COVAR_MDS_UPDRS_III_EP.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --geno 0.05
  --glm a0-ref
  --hwe 0.0001
  --mac 2
  --maf 0.01
  --out /home/jupyter/COMT_AMPPD/MDS_UPDRS_III_COMT_All_variants
  --pheno /home/jupyter/COMT_AMPPD/COVAR_MDS_UPDRS_III_EP.txt
  --pheno-name MDS_UPDRS_III

Start time: Tue Apr  1 14:52:11 2025
7445 MiB RAM detected, ~6057 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.fam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.bim.
1 quantitative phenotype loade

In [109]:
glm_UPDRSIII_AllVariants = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_III_COMT_All_variants.MDS_UPDRS_III.glm.linear', delim_whitespace=True)
glm_UPDRSIII_AllVariants_ADD = glm_UPDRSIII_AllVariants[glm_UPDRSIII_AllVariants['TEST']=="ADD"]

In [110]:
# Save the results of the glm to a file
glm_UPDRSIII_AllVariants_ADD.to_csv(f'{WORK_DIR}/UPDRSIV_COMT.AllVariants.csv')

In [108]:
# Also check the P corrected for multiple comparison

glm_UPDRSIII_adj = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_III_COMT_All_variants.MDS_UPDRS_III.glm.linear.adjusted', delim_whitespace=True)

## MDS UPDRS IV

In [ ]:
# Download clinical data: MDS-UPDRS scale part III
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {AMP_CLINICAL_RELEASE_PATH}/MDS_UPDRS_Part_IV.csv {WORK_DIR}')

In [108]:
# Read the MDS_UPDRS_PART_IV.csv file and 
# format it to only contain baseline and the selected columns
mds_updrs_iv = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_Part_IV.csv')
mds_updrs_iv = mds_updrs_iv[['participant_id','visit_month','mds_updrs_part_iv_summary_score']]
mds_updrs_iv_m0 = mds_updrs_iv[mds_updrs_iv['visit_month']==0] 
mds_updrs_iv_m0.columns = ['IID','visit_month','MDS_UPDRS_IV']

In [111]:
# Merge with the COV file and check the sample size
AMPPD_COV_UPDRSIV= pd.merge(mds_updrs_iv_m0,AMPPD_COV,on="IID")

In [112]:
# Select columns
AMPPD_COV_UPDRSIV = AMPPD_COV_UPDRSIV[['IID','IID','AGE','PHENO','SEX','MDS_UPDRS_IV','PC1','PC2','PC3','PC4','PC5']]

In [113]:
# Rename columns
AMPPD_COV_UPDRSIV.columns = ['FID','IID','AGE','PHENO','SEX','MDS_UPDRS_IV','PC1','PC2','PC3','PC4','PC5']

In [114]:
# Reformat the interest column to numeric
AMPPD_COV_UPDRSIV["MDS_UPDRS_IV"] = pd.to_numeric(AMPPD_COV_UPDRSIV["MDS_UPDRS_IV"])

In [115]:
# Split the data on PD and HC
AMPPD_COV_UPDRSIV_EP=AMPPD_COV_UPDRSIV[AMPPD_COV_UPDRSIV['PHENO']==2]

In [ ]:
# Check NAs and fill with -9
AMPPD_COV_UPDRSIV_EP['MDS_UPDRS_V'] = AMPPD_COV_UPDRSIV_EP['MDS_UPDRS_IV'].fillna(-9)

In [119]:
# Save to a file
AMPPD_COV_UPDRSIV_EP.to_csv(f'{WORK_DIR}/COVAR_MDS_UPDRS_IV_EP.txt',sep="\t",index=None)

In [47]:
# Test the association for all variants of the gene detected with UPDRSIV scale total score

WORK_DIR = f'/home/jupyter/COMT_AMPPD'

! /home/jupyter/tools/plink2 \
--bfile {WORK_DIR}/COMT \
--glm a0-ref\
--pheno {WORK_DIR}/COVAR_MDS_UPDRS_IV_EP.txt \
--pheno-name MDS_UPDRS_IV \
--adjust \
--mac 2 \
--geno 0.05 \
--maf 0.01 \
--hwe 0.0001 \
--covar {WORK_DIR}/COVAR_MDS_UPDRS_IV_EP.txt \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5 \
--covar-variance-standardize \
--out {WORK_DIR}/MDS_UPDRS_IV_COMT_All_variants

PLINK v2.0.0-a.6.9LM AVX2 AMD (29 Jan 2025)        cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/COMT_AMPPD/MDS_UPDRS_IV_COMT_All_variants.log.
Options in effect:
  --adjust
  --bfile /home/jupyter/COMT_AMPPD/COMT
  --covar /home/jupyter/COMT_AMPPD/COVAR_MDS_UPDRS_IV_EP.txt
  --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5
  --covar-variance-standardize
  --geno 0.05
  --glm a0-ref
  --hwe 0.0001
  --mac 2
  --maf 0.01
  --out /home/jupyter/COMT_AMPPD/MDS_UPDRS_IV_COMT_All_variants
  --pheno /home/jupyter/COMT_AMPPD/COVAR_MDS_UPDRS_IV_EP.txt
  --pheno-name MDS_UPDRS_IV

Start time: Tue Apr  1 14:52:18 2025
7445 MiB RAM detected, ~6053 available; reserving 3722 MiB for main workspace.
Using up to 2 compute threads.
5086 samples (2283 females, 2803 males; 5086 founders) loaded from
/home/jupyter/COMT_AMPPD/COMT.fam.
502 variants loaded from /home/jupyter/COMT_AMPPD/COMT.bim.
1 quantitative phenotype loaded (17

In [123]:
glm_UPDRSIV_AllVariants = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_IV_COMT_All_variants.MDS_UPDRS_IV.glm.linear', delim_whitespace=True)
glm_UPDRSIV_AllVariants_ADD= glm_UPDRSIV_AllVariants[glm_UPDRSIV_AllVariants['TEST']=="ADD"]

In [125]:
# Save the results of the glm to a file
glm_UPDRSIV_AllVariants_ADD.to_csv(f'{WORK_DIR}/UPDRSIV_COMT.AllVariants.csv')

In [126]:
# Also check the P corrected for multiple comparison

glm_UPDRSIV_AllVariants_adj = pd.read_csv(f'{WORK_DIR}/MDS_UPDRS_IV_COMT_All_variants.MDS_UPDRS_IV.glm.linear.adjusted', delim_whitespace=True)